In [1]:
import os
import json
import sys

import pandas as pd
import numpy as np
import yaml

APP_ROOT_DIR = os.getenv('AI_APP')
ROOT_DIR = os.getenv('AI_HOME')
BASE_PATH_CFG = os.getenv('AI_CFG')
MODEL_PATH = "/aimldl-cfg/model/"

if APP_ROOT_DIR not in sys.path:
  sys.path.append(APP_ROOT_DIR)

import common

/home/alpha/virtualmachines/virtualenvs/py_3-6-5_2018-11-21/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/alpha/virtualmachines/virtualenvs/py_3-6-5_2018-11-21/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
csv_path = "/aimldl-dat/logs/mask_rcnn/evaluate-mask_rcnn.csv"
# csv_path = "/aimldl-dat/logs/mask_rcnn/evaluate_combined.csv"
timestamp = common.timestamp()
AI_REPORTS_DIR = os.getenv('AI_REPORTS')
dnn_name = "mask_rcnn"
rptfilename = "rpt-evaluate-{}-{}.{}".format(dnn_name,timestamp,'csv')
report_filepath = os.path.join(AI_REPORTS_DIR, dnn_name, rptfilename)
print(report_filepath)

/aimldl-rpt/mask_rcnn/rpt-evaluate-mask_rcnn-281119_145843.csv


In [3]:
exp_df = pd.read_csv(csv_path)
exp_df.dropna(inplace=True)
exp_df.reset_index(inplace=True)
exp_df.drop(['sno', 'index'], axis=1, inplace=True)

In [4]:
summary_json_arr = [exp_df['rpt_summary_filepath'][i] for i in range(len(exp_df))]
rpt_json_arr = [exp_df['rpt_metric_filepath'][i] for i in range(len(exp_df))]
data_df = pd.DataFrame(columns = ['timestamp', 'dataset', 'expid', 'modelinfocfg', 'weights_path', 'class_names_dataset', 'subset', 'total_images', 'gt_total_annotation', 'pred_total_annotation', 'mean_pred_match_total_annotation', 'total_execution_time', 
                                'TP/(TP+FN)', 'TP/(TP+FP+FN)', 'mAP', 'f1', 'mAP_non_zero_images', 'images_recall', 'false_pos', 'false_neg', 'precision', 'recall', 'total_images_with_pred', 'total_image_without_pred'])

In [40]:
i=1
rpt_path = rpt_json_arr[i]
summary_path = summary_json_arr[i]
rpt_df = pd.read_json(rpt_path)

j=1
print(rpt_df.keys())
print("ap_per_image: {}".format(rpt_df['ap_per_image'][j]))

print("precisions: {}".format(rpt_df['precisions'][j]))
print("recalls: {}".format(rpt_df['recalls'][j]))
print("recall_bbox: {}".format(rpt_df['recall_bbox'][j]))
print("pred_match_scores: {}".format(rpt_df['pred_match_scores'][j]))
print("gt_match: {}".format(rpt_df['gt_match'][j]))
print("pred_match: {}".format(rpt_df['pred_match'][j]))
print("overlaps_mask_iou: {}".format(rpt_df['overlaps_mask_iou'][j]))
print("pred_match_class_names: {}".format(rpt_df['pred_match_class_names'][j]))

# mAP_per_image = np.array([ np.mean(i) for i in list(rpt_df['ap_per_image']) ])
# mAP_per_image_ones = np.array([ np.mean(i) for i in list(np.where(rpt_df['ap_per_image']==1.)[0]) ])
# print(mAP_per_image_ones)
# print(mAP_per_image)

Index(['ap_per_image', 'f1_per_image', 'gt_match', 'iou_thresholds',
       'overlaps_mask_iou', 'positive_ids_bbox', 'precisions', 'pred_match',
       'pred_match_class_ids', 'pred_match_class_names', 'pred_match_scores',
       'pred_match_total_annotation', 'recall_bbox', 'recalls'],
      dtype='object')
ap_per_image: [0.833333333333333, 0.833333333333333, 0.833333333333333, 0.833333333333333, 0.833333333333333, 0.833333333333333, 0.833333333333333, 0.833333333333333, 0.5, 0.0]
precisions: [[1.0, 0.6666666666666661, 0.6666666666666661, 0.5, 0.4], [1.0, 0.6666666666666661, 0.6666666666666661, 0.5, 0.4], [1.0, 0.6666666666666661, 0.6666666666666661, 0.5, 0.4], [1.0, 0.6666666666666661, 0.6666666666666661, 0.5, 0.4], [1.0, 0.6666666666666661, 0.6666666666666661, 0.5, 0.4], [1.0, 0.6666666666666661, 0.6666666666666661, 0.5, 0.4], [1.0, 0.6666666666666661, 0.6666666666666661, 0.5, 0.4], [1.0, 0.6666666666666661, 0.6666666666666661, 0.5, 0.4], [1.0, 0.5, 0.33333333333333304, 0.25, 0.2],

In [5]:
for i in range(len(exp_df)):
    rpt_path = rpt_json_arr[i]
    summary_path = summary_json_arr[i]
    rpt_df = pd.read_json(rpt_path)
    mAP_per_image = np.array([ np.mean(i) for i in list(rpt_df['ap_per_image']) ])
    mAP_per_image_ones = np.array([ np.mean(i) for i in list(np.where(rpt_df['ap_per_image']==1.)) ])
    

    len_iou = len(rpt_df['iou_thresholds'][i])
    sum_pred_match = np.zeros(len_iou, dtype=int)
    for i in list(rpt_df['pred_match_total_annotation']):
        sum_pred_match = sum_pred_match + i
    mean_pred_match_total_annotation = np.ceil(np.mean(sum_pred_match))

    mAP = np.mean(mAP_per_image)
    f1 = np.mean([ np.mean(i) for i in list(rpt_df['f1_per_image']) ])

    images_with_non_zero_mAP = mAP_per_image[np.where(mAP_per_image>0.)]
    mAP_non_zero_images = np.mean(images_with_non_zero_mAP)
    total_images_with_pred = len(images_with_non_zero_mAP)*1.
    total_image_without_pred = len(mAP_per_image)*1. - total_images_with_pred
    images_recall = (total_images_with_pred/len(mAP_per_image)*1.)*100
    
    with open(summary_path) as json_data:
        sum_data = json.load(json_data)
    
    model_file_name = sum_data['model']
    model_file_path = MODEL_PATH+model_file_name
    with open(model_file_path) as file:
        doc = yaml.safe_load(file)
    weights_path = doc['weights_path']
    
    dataset = sum_data['dataset']
    expid = sum_data['arch']
    modelinfocfg = sum_data['model']
    
    class_names_dataset = sum_data['class_names_dataset'][1:]
    class_names_dataset = (';').join(class_names_dataset)
    total_execution_time = sum_data['total_execution_time']
    gt_total_annotation = sum_data['gt_total_annotation']
    pred_total_annotation = sum_data['pred_total_annotation']
    
    false_pos= np.subtract(pred_total_annotation, mean_pred_match_total_annotation)
    false_neg = np.subtract(gt_total_annotation, mean_pred_match_total_annotation)
    precision = mean_pred_match_total_annotation/(mean_pred_match_total_annotation + false_pos)
    recall = mean_pred_match_total_annotation/(mean_pred_match_total_annotation + false_neg)
    
    acc_1 = mean_pred_match_total_annotation/(mean_pred_match_total_annotation + false_neg)
    acc_2 = mean_pred_match_total_annotation/(mean_pred_match_total_annotation + false_neg + false_pos)

    total_images = sum_data['total_images']
    subset = sum_data['subset']
    
    data_df = data_df.append({'timestamp':timestamp, 'dataset':dataset, 'expid':expid, 'modelinfocfg':modelinfocfg, 'weights_path':weights_path, 'class_names_dataset':class_names_dataset, 'subset':subset, 'total_images':total_images, 
                        'gt_total_annotation':gt_total_annotation, 'pred_total_annotation':pred_total_annotation, 'mean_pred_match_total_annotation':mean_pred_match_total_annotation, 
                        'total_execution_time':total_execution_time, 'TP/(TP+FN)':acc_1, 'TP/(TP+FP+FN)':acc_2, 'mAP':mAP, 'f1':f1, 'mAP_non_zero_images':mAP_non_zero_images, 'images_recall':images_recall, 'false_pos':false_pos , 
                        'false_neg':false_neg , 'precision':precision , 'recall':recall , 'total_images_with_pred':total_images_with_pred, 'total_image_without_pred':total_image_without_pred}, ignore_index=True)


In [7]:
data_df

,timestamp,dataset,expid,modelinfocfg,weights_path,class_names_dataset,subset,total_images,gt_total_annotation,pred_total_annotation,...,mAP,f1,mAP_non_zero_images,images_recall,false_pos,false_neg,precision,recall,total_images_with_pred,total_image_without_pred
0,281119_145843,PXL-151119_175327,evaluate-97591269-f016-4a66-9467-823fa1d2b834,vidteq-hmd-151119_200020-mask_rcnn.yml,mask_rcnn/151119_195609/train_hmd_151119_20002...,signage;traffic_light;traffic_sign,val,4913,13905,17375,...,0.425192,1.301382,0.478572,88.845919,11391.0,7921.0,0.344403,0.430349,4365.0,548.0
1,281119_145843,PXL-151119_175327,evaluate-8390df7a-da12-4d8f-bc32-1f1beaf66981,vidteq-hmd-181119_163226-mask_rcnn.yml,mask_rcnn/181119_162637/train_hmd_181119_16322...,signage;traffic_light;traffic_sign,val,4913,13907,14250,...,0.377530,1.016027,0.447588,84.347649,9093.0,8750.0,0.361895,0.370820,4144.0,769.0
2,281119_145843,PXL-151119_175327,evaluate-52f8d1ce-fcd2-416e-b753-1cad073b778e,vidteq-hmd-191119_132924-mask_rcnn.yml,mask_rcnn/191119_105551/train_hmd_191119_13292...,signage;traffic_light;traffic_sign,val,4913,13907,16828,...,0.400935,1.194380,0.457241,87.685732,11347.0,8426.0,0.325707,0.394118,4308.0,605.0
3,281119_145843,PXL-151119_175327,evaluate-9e3e151d-bb1e-41cb-b38a-8c6e63db91c2,vidteq-hmd-201119_184448-mask_rcnn.yml,mask_rcnn/201119_181215/train_hmd_201119_18444...,signage;traffic_light;traffic_sign,val,4913,13907,16035,...,0.402491,1.161904,0.461695,87.176878,10560.0,8432.0,0.341441,0.393687,4283.0,630.0
4,281119_145843,PXL-151119_175327,evaluate-a0f58f78-b31c-4d27-bc5f-d69a7d9208b2,vidteq-hmd-1-mask_rcnn.yml,vidteq/hmd/1/mask_rcnn/weights/mask_rcnn_hmd_0...,signage;traffic_light;traffic_sign,val,4913,13907,3002,...,0.115485,0.173506,0.331411,34.846326,1696.0,12601.0,0.435043,0.093910,1712.0,3201.0


In [8]:
data_df[['uuid', 'eval_on', 'username', 'aids_db_name', 'expid',
       'prog_log', 'iou', 'rpt_imagelist_filepath', 'rpt_metric_filepath',
       'rpt_summary_filepath']] = exp_df[['uuid', 'eval_on', 'username', 'aids_db_name', 'expid',
       'prog_log', 'iou', 'rpt_imagelist_filepath', 'rpt_metric_filepath',
       'rpt_summary_filepath']]

In [9]:
data_df.sort_values(by=['class_names_dataset', 'subset'], inplace=True)

In [10]:
data_df.reset_index(inplace=True)

In [11]:
data_df.drop(['index'], axis=1, inplace=True)

In [12]:
data_df.to_csv(report_filepath)

In [108]:
# import os
# train_logpath = "/aimldl-dat/logs/mask_rcnn/train.output-201119_184439.log"

# cmd = "".format(train_logpath)
# print(cmd)
# os.system("grep 'step - ' "+train_logpath+" | tail -1 | cut -d'-' -f3-")

In [14]:
x = pd.read_json(rpt_json_arr[0])

In [15]:
x

,ap_per_image,f1_per_image,gt_match,iou_thresholds,overlaps_mask_iou,positive_ids_bbox,precisions,pred_match,pred_match_class_ids,pred_match_class_names,pred_match_scores,pred_match_total_annotation,recall_bbox,recalls
0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[0.5, 0.55, 0.6000000000000001, 0.65, 0.700000...","[[[0.8327525854110711]], [[0.8327525854110711]...","[[0], [0], [0], [0], [0], [0], [0], [], [], []]","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[3], [3], [3], [3], [3], [3], [3], [], [], []]","[[traffic_sign], [traffic_sign], [traffic_sign...","[[0.9996556043624871], [0.9996556043624871], [...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."
1,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6111111243565...","[5.084548805505889, 5.084548805505889, 5.08454...","[[2.0, 4.0, 3.0, 1.0, 5.0, 0.0], [2.0, 4.0, 3....","[0.5, 0.55, 0.6000000000000001, 0.65, 0.700000...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.8372092843055721...","[[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5], [0, 1...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.857142857142...","[[5.0, 3.0, 0.0, 2.0, 1.0, 4.0, -1.0], [5.0, 3...","[[2, 1, 1, 2, 2, 1], [2, 1, 1, 2, 2, 1], [2, 1...","[[signage, traffic_light], [signage, traffic_l...","[[0.999624133110046, 0.9992012381553651, 0.989...","[6, 6, 6, 6, 6, 6, 4, 2, 2, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 0.833333333333333, 0...","[[0.16666667163372, 0.33333334326744, 0.5, 0.6..."
2,"[0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.5, ...","[3.683333333333333, 3.683333333333333, 3.68333...","[[-1.0, 3.0, 1.0, 0.0], [-1.0, 3.0, 1.0, 0.0],...","[0.5, 0.55, 0.6000000000000001, 0.65, 0.700000...","[[[0.0, 0.0, 0.0, 0.8287795782089231], [0.0, 0...","[[0, 1, 3], [0, 1, 3], [0, 1, 3], [0, 1, 3], [...","[[1.0, 1.0, 0.75, 0.75, 0.6000000000000001, 0....","[[3.0, 2.0, -1.0, 1.0, -1.0, -1.0], [3.0, 2.0,...","[[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [...","[[signage], [signage], [signage], [signage], [...","[[0.9960655570030211, 0.9396281242370601, -1.0...","[3, 3, 3, 3, 3, 2, 2, 1, 0, 0]","[0.75, 0.75, 0.75, 0.75, 0.5, 0.5, 0.5, 0.5, 0...","[[0.25, 0.5, 0.5, 0.75, 0.75, 0.75], [0.25, 0...."
3,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, ...","[0.6666666666666661, 0.6666666666666661, 0.666...","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[0.5, 0.55, 0.6000000000000001, 0.65, 0.700000...","[[[0.0], [0.810043692588806]], [[0.0], [0.8100...","[[1], [1], [1], [1], [1], [1], [1], [1], [], []]","[[0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5, 0.5...","[[-1.0, 0.0], [-1.0, 0.0], [-1.0, 0.0], [-1.0,...","[[3], [3], [3], [3], [3], [3], [3], [], [], []]","[[traffic_sign], [traffic_sign], [traffic_sign...","[[-1.0, 0.9972258210182191], [-1.0, 0.99722582...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ...","[[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-1.0], [-1.0], [-1.0], [-1.0], [-1.0], [-1.0...","[0.5, 0.55, 0.6000000000000001, 0.65, 0.700000...","[[], [], [], [], [], [], [], [], [], []]","[[], [], [], [], [], [], [], [], [], []]","[[], [], [], [], [], [], [], [], [], []]","[[], [], [], [], [], [], [], [], [], []]","[[], [], [], [], [], [], [], [], [], []]","[[], [], [], [], [], [], [], [], [], []]","[[], [], [], [], [], [], [], [], [], []]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[], [], [], [], [], [], [], [], [], []]"
5,"[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[2.566666666666666, 2.566666666666666, 2.56666...","[[0.0], [0.0], [0.0], [0.0], [0.0], [-1.0], [-...","[0.5, 0.55, 0.6000000000000001, 0.65, 0.700000...","[[[0.7163120508193971], [0.0], [0.0], [0.0]], ...","[[0], [0], [0], [0], [0], [], [], [], [], 

In [16]:
x['ap_per_image'][0]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]

In [18]:
np.mean(x['ap_per_image'][2])

0.45625